In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("data/Jewelry_Dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/Jewelry_Dataset.csv'

In [8]:
import os

In [16]:
print(os.listdir())

['.ipynb_checkpoints', 'JP_Optimization.ipynb']


In [9]:
df = pd.read_csv("data/Jewelry_Dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/Jewelry_Dataset.csv'

In [10]:
import os

In [11]:
os.getcwd()

'C:\\Users\\hp\\Desktop\\Jewelry_Price_Optimization\\notebooks'

In [12]:
os.listdir("data")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data'

In [13]:
df = pd.read_csv("C:/Users/hp/Desktop/Jewelry_Price_Optimization/data/Jewelry_Dataset.csv")

In [14]:
df.head()

,2018-12-01 11:40:29 UTC,1924719191579951782,1842195256808833386,1,1806829201890738522,jewelry.earring,0,561.51,1515915625207851155,Unnamed: 9,red,gold,diamond
0,2018-12-01 17:38:31 UTC,1924899396621697920,1806829193678291446,1,1.806829e+18,NaN,NaN,212.14,1.515916e+18,NaN,yellow,gold,NaN
1,2018-12-02 13:53:42 UTC,1925511016616034733,1842214461889315556,1,1.806829e+18,jewelry.pendant,1.0,54.66,1.515916e+18,f,white,gold,sapphire
2,2018-12-02 17:44:02 UTC,1925626951238681511,1835566849434059453,1,1.806829e+18,jewelry.pendant,0.0,88.90,1.515916e+18,f,red,gold,diamond
3,2018-12-02 21:30:19 UTC,1925740842841014667,1873936840742928865,1,1.806829e+18,jewelry.necklace,0.0,417.67,1.515916e+18,NaN,red,gold,amethyst
4,2018-12-02 22:09:34 UTC,1925760595336888995,1835566854827934449,1,1.806829e+18,jewelry.earring,1.0,102.27,1.515916e+18,NaN,red,gold,NaN
